# Bucket Brigade - Nested edition

Bucket Brigade is a system for taking a large parquet file breaking it up spatially across multiple files using geohash to organize files into nested directories.

In [ ]:
import os
from collections import Counter
import time
import json

import pandas as pd
import geopandas as gpd
#import dask_geopandas
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
from shapely import wkb, wkt
from shapely.geometry import box
from shapely.geometry import Polygon
import pygeohash as pgh


## What is a GEOHash

Lets take a second to look at geo hash and what it looks like.

In [ ]:
print(pgh.encode(latitude=42.6, longitude=-5.6))
print(pgh.encode(latitude=42.6, longitude=-5.6, precision=5))
print(pgh.encode(latitude=32.0, longitude=-7.0, precision=5))

print(pgh.decode(geohash='ezs42'))
print(pgh.geohash_approximate_distance(geohash_1='bcd3u', geohash_2='bc83n') /1000 ,"km")
#not found in lib???
#print(pgh.get_adjacent(geohash='kd3ybyu', direction='right'))

## Settings

Pick a file and then read it in using GeoPandas

In [ ]:
selected_file = "~/src/project/cmr-bigstac-prototype/bigstac/scripts_explore/3mil_no_global_bounds.parquet"

# Read the GeoParquet file
gdf = gpd.read_parquet(selected_file)

## Some utility functions

going to use these things latter on

In [ ]:
# some utilities
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        #print(f"Successfully wrote to the file {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file {filename}: {e}")

def make_geo_box(sub:str, details:dict):
  """Make a directory to store parquet files in"""
  data = f"{os.getcwd()}/data2/{sub}"
  #print(data)
  if not os.path.exists(data):
    os.makedirs(data)
    write_string_to_file(f"{data}/info.json", json.dumps(details))

def hash_to_box(hash1:str, hash2:str) -> box:
  miny, minx = pgh.decode(hash1)
  maxy, maxx = pgh.decode(hash2)
  return box(minx, miny, maxx, maxy)

def find_hemispheres(bound_obj:Polygon) -> str:
    minx, miny, maxx, maxy = bound_obj.bounds
    #print(minx, miny, maxx, maxy)
    hemispheres = set()

    if minx < 0:
        if miny < 0:
            hemispheres.add("SW")
        if maxy > 0:
            hemispheres.add("NW")
    if maxx > 0:
        if miny < 0:
            hemispheres.add("SE")
        if maxy > 0:
            hemispheres.add("NE")

    if len(hemispheres) == 0:
        # The box is exactly on the equator and prime meridian
        return "Central"
    elif len(hemispheres) == 4:
        return "All"
    else:
        return '-'.join(hemispheres)

#print(find_hemispheres(hash_to_box('9hr', 'f8b')))

def is_global(minx, miny, maxx, maxy):
  if minx < -180 or maxx > 180 or miny < -90 or maxy > 90:
    return True
  else:
    return False

def is_global_from_geohash(hash1, hash2):
  is_global_from_geometry(hash_to_box(hash1, hash2))

def is_global_from_geometry(geometry_obj):
  minx, miny, maxx, maxy = geometry_obj.bounds
  is_global(minx, miny, maxx, maxy)

def hash_to_path(hash1, hash2):
  hash_path = ''
  # test if box fits in a bin
  if hash1 == hash2:
    # exact match, three down
    hash_path = f"{hash1[0]}/{hash1[1]}/{hash1[2]}"
  elif hash1[0] == hash2[0] and hash1[1] == hash2[1]:
    # match 2 layers down
    hash_path = f"{hash1[0]}/{hash1[1]}"
  elif hash1[0] == hash2[0]:
    # match 1 layer down
    hash_path = f"{hash1[0]}"
  elif is_global_from_geohash(hash1, hash2):
    # global
      hash_path = "global"
  else:
    # does not fit in just one box, check hemispheres
    hash_path = find_hemispheres(hash_to_box(hash1, hash2))
  
  return hash_path

In [ ]:
# if your really worried about the code above, run this otherwise minimize it and forget it
def test_find_hemispheres():
  assert find_hemispheres(box(-10, -10, 1, 1)) == 'All'
  assert find_hemispheres(box(-10, -10, -5, -5)) == 'SW'
  assert find_hemispheres(box(5, -10, 10, -5)) == 'SE'
  assert find_hemispheres(box(-10, 5, -5, 10)) == 'NW'
  assert find_hemispheres(box(5, 5, 10, 10)) == 'NE'
  assert find_hemispheres(box(-10, -10, 10, 0)) == 'SE-SW'
  assert find_hemispheres(box(-10, 0, 10, 10)) == 'NE-NW'
  assert find_hemispheres(box(-10, -10, 0, 10)) == 'NW-SW'
  assert find_hemispheres(box(0, -10, 10, 10)) == 'SE-NE'
  assert find_hemispheres(box(-10, 0, 0, 10)) == 'NW'
  assert find_hemispheres(box(0, -10, 10, 0)) == 'SE'
  assert find_hemispheres(box(-10, -10, 0, 0)) == 'SW'
  assert find_hemispheres(box(0, 0, 10, 10)) == 'NE'
  assert find_hemispheres(box(-10, 0, 0, 10)) == 'NW'
  assert find_hemispheres(box(0, -10, 10, 0)) == 'SE'
  assert find_hemispheres(box(-10, -10, 0, 0)) == 'SW'
  assert find_hemispheres(box(0, 0, 10, 10)) == 'NE'
  assert find_hemispheres(box(-10, 0, 0, 10)) == 'NW'
  assert find_hemispheres(box(0, -10, 10, 0)) == 'SE'
test_find_hemispheres()

## Break things up

Here we will go thru all the rows in a parquet file. For each file we will use the bounding box 
(bbox) for the row and calculate the GeoHash for the two corners of that bbox. We will then use those two GeoHash values, which should be 1 character long, to create a 'hash code' for use as the bucket name to store parquet files in with the rows corresponding to that bucket. An example would 
be `4-g`.

Using the lowest precision of 1 will give us 32 grids. From these 32 grids we could have 1024 boxes for every combination of of two bounding box GeoHash codes.

Create a GeoDataFrame for every bucket and concat the record to it.

Each bucket will also have a file called info.json that will contain the details of the bucket.

In [ ]:
# helper functions

def flush_parquet(parquet_data:dict, hash_path:str):
    """ write out parquet files and clear out the value in parquet_data. """
    parquet_file_name = f"{os.getcwd()}/data2/{hash_path}/data0001.parquet"

    print(f"{hash_path}->{parquet_file_name}")
    if os.path.exists(parquet_file_name):
        existing_data = gpd.read_parquet(parquet_file_name)
        clean_data = existing_data.drop(columns=['0', 0], errors='ignore')
        combined_data = pd.concat([clean_data, parquet_data[hash_path]], ignore_index=True)
        combined_data.to_parquet(parquet_file_name, index=False)
    else:
        # If file doesn't exist, write new data
        parquet_data[hash_path].drop(columns=['0', 0], inplace=True, errors='ignore')
        parquet_data[hash_path].to_parquet(parquet_file_name, index=False)
    # Update the parquet_data dictionary with an empty GeoDataFrame
    parquet_data[hash_path] = gpd.GeoDataFrame(columns=parquet_data[hash_path].columns)

def concat_record_to_parquet(parquet_data:dict, hash_path:str, row:pd.Series, columns:pd.Index):
  """ Collect records in a dataframe and flush them out when to many have been stored. """
  if hash_path not in parquet_data:
      parquet_data[hash_path] = gpd.GeoDataFrame(columns=columns)
  new_row = pd.DataFrame([row], columns=columns)
  #print(new_row.iloc[0].geometry.bounds)
  parquet_data[hash_path] = pd.concat([parquet_data[hash_path], new_row], ignore_index=True)
  
  if len(parquet_data[hash_path]) > 10000:
    flush_parquet(parquet_data, hash_path)

### Create the buckets
Do the work to break up the larger parquet file by location. The functions above are helper functions to try to keep the code here small and managable.

Last run this took over 5 hours to run.

In [ ]:
counter = Counter()
parquet_data = {}

partition_start = int(time.time())
for index, row in gdf.iterrows():
    #if index == 0:
    #  print(type(row))
    #  print(type(gdf.columns))
    #if index < 1000:
    #  # skip ahead to some interesting records
    #  continue
    geometry = row['geometry']
    minx, miny, maxx, maxy = geometry.bounds
    hash1 = pgh.encode(latitude=minx, longitude=miny, precision=3)
    hash2 = pgh.encode(latitude=maxx, longitude=maxy, precision=3)
    distance = pgh.geohash_approximate_distance(geohash_1=hash1, geohash_2=hash2)
    bound_hash = f"{hash1}-{hash2}"
    hash_path = hash_to_path(hash1, hash2)
    details = {'bounds': geometry.bounds,
      'hash1': hash1,
      'hash2': hash2,
      'hash': bound_hash,
      'distance': distance}
    make_geo_box(hash_path, details)
    counter[hash_path] += 1

    concat_record_to_parquet(parquet_data, hash_path, row, gdf.columns)
    
    limit_records = 10000 # 100,000 rows
    if len(counter) > limit_records:
      print(f"breaking after {limit_records}")
      break
    elif len(counter) % 1000 == 0:
      print(f"{len(counter)} records processed")
print(counter)
partition_stop = int(time.time())
partition_durration_min = int(partition_stop - partition_start) / 60
print (f"it took {partition_durration}m to partition.")

print('Do not forget to call the final flush below!!!!!')

#### Last
Do not forget to call the final flush

In [ ]:
print("Flush now")
for key, value in parquet_data.items():
  flush_parquet(parquet_data, key)

### Write out buckets

Now that we have a dictinary of data frames, write them all out.

This has been fast, around 5 min.

In [ ]:
c = 1
for key, value in parquet_data.items():
    print(f"writing {key} to disk. {c} of {len(parquet_data.keys())}")
    parquet_data[key].to_parquet(f"{os.getcwd()}/data/{key}/{key}.parquet")
    c += 1

## Clean up
Run the following code to remove things

In [ ]:
# just remove the parquet files
#!find data2 -name '*.parquet' -exec rm {} \;

# remove the data directory
#!rm -rf data2

# Data Investigation
Now that there is data, lets have a look at it by first picking a box and then looking at the data in that box.

In [ ]:
import geopandas as gpd
from shapely.geometry import box
import duckdb

def find_files_with_extension(path, extension):
    return [
        os.path.join(root, file)
        for root, dirs, files in os.walk(path)
        for file in files
        if file.endswith(extension)
    ]

def geohashes(boundy_box:box) -> (str, str):
  minx, miny, maxx, maxy = boundy_box.bounds
  p1 = pgh.encode(latitude=miny, longitude=minx, precision=3)
  p2 = pgh.encode(latitude=maxy, longitude=maxx, precision=3)
  return p1, p2

# Create a shapely box geometry
usa = box(-124.848974, 24.396308, -66.885444, 49.384358) # north
colorado = box(-109.060251, 36.992424, -102.040899, 41.003444) # one deep
rhode_island = box(-71.907258, 41.146339, -71.088571, 42.018798) # 2 deep
maryland = box(-79.4871, 37.9130, -75.0491, 39.7229) # one deep
baltimore_county = box(-76.711607, 39.214233, -76.347517, 39.721644)
baltimore = box(-76.711607, 39.197233, -76.529453, 39.372069) # 3 deep
virgina_part = box(-78, 37, -77, 39) # 2 deep

# Create a GeoDataFrame
usa_bbox = gpd.GeoDataFrame({'name': ['Continental USA']}, 
                            geometry=[usa], 
                            crs="EPSG:4326")
# Print the GeoDataFrame
#print(usa_bbox)

p = lambda c, b: print(f"{c:5}: {geohashes(b)}-> {hash_to_path(*geohashes(b))}")

p("USA", usa)
p("MD", maryland)
p("Virg", virgina_part)
p("Balt", baltimore)

sub = hash_to_path(*geohashes(maryland))
search_path = f"{os.getcwd()}/data2/{sub}"
file_list = find_files_with_extension(search_path, ".parquet")
print(f"maryland file count: {len(file_list)}")
file_list_str = ", ".join(file_list)

sql = f'''SELECT COUNT(*) FROM parquet_scan({file_list})'''
print(sql)
duckdb.sql(sql)
